## Model Tuning with GridSearch

In [ ]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

## Load Data

In [ ]:
# Read in the banking data set.
df = pd.read_csv("../../0_data/banking/bank-additional-full.csv", sep=";")
df.head(3)

In [ ]:
# Split features and labels.
X = df.drop(columns=["duration", "pdays", "y"])
y = df["y"].replace({"yes": 1, "no": 0})

## Create the model

In [ ]:
# Get column names per data type.
categorical = X.select_dtypes("object").columns
numerical = X.select_dtypes("number").columns

In [ ]:
# Set up the ColumnTransformer.
transformer = ColumnTransformer(
    transformers=[
        ("encode_categorical", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical),
        ("scale_numerical", StandardScaler(), numerical),
    ]
)

In [ ]:
# RandomForest model.
hgb = Pipeline(
    steps=[
        ("preparation", transformer),
        ("model", HistGradientBoostingClassifier()),
    ]
)

## Cross validation

In [ ]:
metrics = {
    "fold": [],
    "accuracy": [],
    "precision": [],
    "recall": []
}

# Perform cross-validation with 5 splits.
kfold = KFold(n_splits=5, shuffle=True)
for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y), start=1):
    
    # Create data sets for the fold.
    # Note: Must use .iloc[] because we have indices!
    X_train = X.iloc[train_idx]
    y_train = y.iloc[train_idx]
        
    X_test = X.iloc[test_idx]
    y_test = y.iloc[test_idx]
        
    # Train the model and get predictions
    hgb.fit(X_train, y_train)
    predictions = hgb.predict(X_test)
        
    # Compute and store performance metrics
    metrics["fold"].append(fold)
    metrics["accuracy"].append(accuracy_score(y_test, predictions))
    metrics["precision"].append(precision_score(y_test, predictions))
    metrics["recall"].append(recall_score(y_test, predictions))

pd.DataFrame(metrics)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define the parameter search space.
# Note: As we have multiple Pipeline steps, use: <step>__<parameter>.
parameters = {
    "model__max_iter": [50, 100, 250],
    "model__learning_rate": [0.1, 0.3, 0.5],
}

In [ ]:
# Set up KFold
kfold = KFold(n_splits=5, shuffle=True)

# Create GridSearch
gs = GridSearchCV(
    hgb,
    param_grid=parameters,
    scoring="recall",
    cv=kfold,
    n_jobs=-1
)

In [ ]:
# Perform GridSearch on the data.
gs.fit(X, y)

In [ ]:
# Display the GridSearch results (as DataFrame).
pd.DataFrame(gs.cv_results_)

In [ ]:
# Best parameter settings
gs.best_params_

In [ ]:
# Best estimator instance
gs.best_estimator_